## Dependencies

In [298]:
import pandas as pd 
import pandas_ta as ta
import numpy as np
from dotenv import load_dotenv
from sklearn.preprocessing import StandardScaler
import os
import requests
from datetime import datetime, timedelta
import re
import time
import warnings
import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from pybit.unified_trading import HTTP
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque
from binance.spot import Spot
import logging
import matplotlib.pyplot as plt


pio.renderers.default = 'browser'
warnings.filterwarnings('ignore')
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)

load_dotenv()

True

## Fetching and Merging Raw Data

In [304]:

class DataManager:
    
    def __init__(self):
        self.glassnode_api_key = os.getenv('GLASSNODE_API_KEY')
        self.bybit_api_key = os.getenv('BYBIT_API_KEY')
        self.bybit_api_secret = os.getenv('BYBIT_API_SECRET')
        self.bybit_session = HTTP(testnet=False, api_key=self.bybit_api_key, api_secret=self.bybit_api_secret)
        self.binance_session = Spot()
        logging.basicConfig(level=logging.INFO)

    @staticmethod
    def datetime_to_unix(date):
        """Convert a datetime object to a Unix timestamp."""
        return int(date.timestamp())

    def _fetch_glassnode_data(self, endpoint, start, end, frequency):
        params = {
            'a': 'BTC',
            's': self.datetime_to_unix(start),
            'u': self.datetime_to_unix(end),
            'i': frequency,
            'f': 'JSON',
            'api_key': self.glassnode_api_key  
        }
        
        base_url = 'https://api.glassnode.com/v1/metrics'
        url = f"{base_url}/{endpoint}"
        name = re.search(r'/([^/]*)$', endpoint).group(1)

        for attempt in range(3):
            try:
                response = requests.get(url, params=params)
                
                if response.status_code == 200:
                    data = response.json()
                    df = pd.DataFrame(data)
                    if 't' in df.columns:
                        df['t'] = pd.to_datetime(df['t'], unit='s')
                    df.rename(columns={'v': name}, inplace=True)
                    logging.info(f"Successfully fetched data for endpoint: {endpoint}")
                    return df
                else:
                    logging.error(f"Failed to fetch data: {response.status_code} - {response.text}")
            except Exception as e:
                logging.error(f"Attempt {attempt + 1}: Exception occurred while fetching data: {e}")

        raise Exception(f"Failed to fetch data after 3 attempts for endpoint: {endpoint}")
    
    def _fetch_and_merge_glassnode_data(self, endpoints, start, end, frequency):
        data_frames = []

        for endpoint in endpoints:
            try:
                df = self._fetch_glassnode_data(endpoint, start, end, frequency)
                if df is not None and not df.empty:
                    data_frames.append(df.set_index('t'))
                else:
                    logging.warning(f"No data found for endpoint: {endpoint}")
            except Exception as e:
                logging.error(f"Failed to fetch data for endpoint: {endpoint} with error: {e}")

        if data_frames:
            merged_df = pd.concat(data_frames, axis=1, join='outer')
            merged_df.reset_index(inplace=True)
            merged_df.set_index('t', inplace=True)
            logging.info("Successfully merged data from all endpoints.")
            return merged_df
        else:
            logging.warning("No data frames to merge.")
            return None
    
    def get_trigger_data(self, start, end, frequency='1h'):
        short_term_endpoints = [
            os.getenv('BTC_PRICE'),
            os.getenv('SSR'),
            os.getenv('SUPPLY_IN_PROFIT')
        ]
        return self._fetch_and_merge_glassnode_data(short_term_endpoints, start, end, frequency)

    def get_context_data(self, start, end, frequency='24h'):
        contextual_endpoints = [
            os.getenv('BTC_PRICE'),
            os.getenv('BTC_REALIZED_PRICE'),
            os.getenv('PUELL_MULTIPLE'),
            os.getenv('MVRV_Z_SCORE'),
            os.getenv('ENTITY_ADJ_NUPL'),
            os.getenv('ENTITY_ADJ_DORMANCY_FLOW'),
            os.getenv('SUPPLY_IN_PROFIT')
        ]
        return self._fetch_and_merge_glassnode_data(contextual_endpoints, start, end, frequency)

    def get_bybit_data(self, symbol='BTCUSDT', interval=60, start_time=None, end_time=None):
        try:
            start_time_unix = self.datetime_to_unix(start_time) * 1000 if start_time else None
            end_time_unix = self.datetime_to_unix(end_time) * 1000 if end_time else None
            all_data = []
            fetched_rows = 0

            while end_time_unix > start_time_unix:
                params = {
                    'category': 'spot',
                    'symbol': symbol,
                    'interval': interval,
                    'start': start_time_unix,
                    'end': end_time_unix,
                    'limit': 1000
                }

                response = self.bybit_session.get_kline(**params)

                if response['retCode'] == 0:
                    data = response['result']['list']
                    if not data:
                        logging.info("No more data returned.")
                        break

                    df = pd.DataFrame(data, columns=['start_time', 'open', 'high', 'low', 'close', 'volume', 'turnover'])
                    df['start_time'] = pd.to_datetime(df['start_time'], unit='ms')
                    all_data.append(df)
                    fetched_rows += len(df)

                    logging.info(f"Fetched {len(df)} rows, total fetched: {fetched_rows}")

                    if len(df) < 1000:
                        logging.info("Fetched less than 1000 rows, ending loop.")
                        break  # All data within the range has been retrieved

                    # Update end_time_unix for the next call to avoid overlapping data
                    earliest_timestamp = df['start_time'].iloc[-1].timestamp() * 1000
                    end_time_unix = int(earliest_timestamp) - (interval * 60 * 1000)  # Decrement by interval in milliseconds
                    logging.info(f"Updated end_time_unix to {end_time_unix} for the next API call.")

                    # Wait for 5 seconds before the next API call
                    time.sleep(1)

                else:
                    logging.error(f"Failed to fetch ByBit data: {response['retMsg']}")
                    break

            if all_data:
                # Concatenate all data frames
                final_df = pd.concat(all_data).drop_duplicates().sort_index()
                final_df.set_index('start_time', inplace=True)
                final_df.drop(columns='volume', inplace=True)
                final_df = final_df.astype(float)  # Convert all columns to float
                #logging.info(f"Successfully fetched {fetched_rows} rows of ByBit data.")
                return final_df
            else:
                logging.warning("No data was fetched from ByBit.")
                return None

        except Exception as e:
            logging.error(f"Exception occurred while fetching ByBit data: {e}")
            return None

    def get_binance_data(self, symbol='BTCUSDT', interval='1h', start_time=None, end_time=None):
            start_time_unix = self.datetime_to_unix(start_time) * 1000 if start_time else None
            end_time_unix = self.datetime_to_unix(end_time) * 1000 if end_time else None
            all_data = []
            fetched_rows = 0

            while start_time_unix < end_time_unix:
                klines = self.binance_session.klines(
                    symbol=symbol,
                    interval=interval,
                    startTime=start_time_unix,
                    endTime=end_time_unix,
                    limit=1000
                )

                if not klines:
                    logging.info("No more data returned.")
                    break

                data = []
                for kline in klines:
                    data.append({
                        'start_time': pd.to_datetime(kline[0], unit='ms'),
                        'open': float(kline[1]),
                        'high': float(kline[2]),
                        'low': float(kline[3]),
                        'close': float(kline[4]),
                        'volume': float(kline[5])
                    })

                df = pd.DataFrame(data)
                all_data.append(df)
                fetched_rows += len(df)

                logging.info(f"Fetched {len(df)} rows, total fetched: {fetched_rows}")

                if len(df) < 1000:
                    logging.info("Fetched less than 1000 rows, ending loop.")
                    break  # All data within the range has been retrieved

                # Update start_time_unix for the next call to avoid overlapping data
                last_timestamp = df['start_time'].iloc[-1].timestamp() * 1000
                start_time_unix = int(last_timestamp) + (3600 * 1000)  # Increment by 1 hour in milliseconds
                #logging.info(f"Updated start_time_unix to {start_time_unix} for the next API call.")

                # Wait for 5 seconds before the next API call
                time.sleep(1)

            if all_data:
                # Concatenate all data frames
                final_df = pd.concat(all_data).drop_duplicates().sort_index()
                final_df.set_index('start_time', inplace=True)
                final_df.drop(columns='volume', inplace=True)
                final_df = final_df.astype(float)  # Convert all columns to float
                #logging.info(f"Successfully fetched {fetched_rows} rows of Binance data.")
                return final_df
            else:
                logging.warning("No data was fetched from Binance.")
                return None



In [ ]:
data_manager = DataManager()

start_date = datetime.strptime('2021-10-01', '%Y-%m-%d')
end_date = datetime.today()

binance_data = data_manager.get_binance_data(symbol='BTCUSDT', interval='1h', start_time=start_date, end_time=end_date)
bybit_data = data_manager.get_bybit_data(symbol='BTCUSDT', start_time=start_date, end_time=end_date)

In [305]:
class TradingStrategy:

    def __init__(self, data_manager):
        self.data_manager = data_manager

    def compute_triggers(self, start, end):
        start = datetime.strptime(start, '%Y-%m-%d')
        end = datetime.strptime(end, '%Y-%m-%d')
        
        # Retrieve trigger data and ByBit data
        trigger_data = self.data_manager.get_trigger_data((start - timedelta(hours=8640)), end)
        binance_data = self.data_manager.get_binance_data(symbol='BTCUSDT', start_time=(start - timedelta(hours=8640/2)), end_time=end)
        binance_data.rename_axis('t', inplace=True)
        
        
        # Merge ByBit data with trigger data
        if binance_data is not None:
            trigger_data = trigger_data.merge(binance_data, left_index=True, right_index=True, how='outer')
        
        # Calculate indicators
        trigger_data['rsi_ssr_smoothed'] = ta.ema(ta.rsi(trigger_data['ssr_oscillator'], length=336), length=800)
        trigger_data['rsi_ssr_smoothed_median'] = trigger_data['rsi_ssr_smoothed'].rolling(window=240).median()
        
        # Clean up and filter data
        trigger_data.drop(columns=['ssr_oscillator', 'profit_relative', 'price_usd_close'], inplace=True)
        trigger_data = trigger_data[trigger_data.index >= start]
        
        return trigger_data

    def compute_context(self, start, end):
        
        start = datetime.strptime(start, '%Y-%m-%d')
        end = datetime.strptime(end, '%Y-%m-%d')
        
        # Retrieve context data
        context_data = self.data_manager.get_context_data((start - timedelta(days=360)), end)

        # Market condition calculations
        context_data['28d_mkt_gradient'] = (context_data['price_usd_close'].diff(28) - context_data['price_realized_usd'].diff(28) - 
                                            (context_data['price_usd_close'].diff(28) - context_data['price_realized_usd'].diff(28)).expanding().mean()) / \
                                            (context_data['price_usd_close'].diff(28) - context_data['price_realized_usd'].diff(28)).expanding().std()
        
        context_data['mayer_multiple'] = context_data['price_usd_close'] / context_data['price_usd_close'].rolling(200).mean()
        context_data['price_profit_corr'] = context_data['price_usd_close'].rolling(7).corr(context_data['profit_relative'])

        # Detect market tops and bottoms
        context_data['top_detection'] = (np.where(context_data['mvrv_z_score'] > 3.8, 1, 0) * 
                                         np.where(context_data['mayer_multiple'] >= 1.3, 1, 0) *
                                         np.where(context_data['net_unrealized_profit_loss_account_based'] >= 0.6, 1, 0) *
                                         np.where(context_data['28d_mkt_gradient'] >= 7, 1, 0)) * context_data['price_usd_close']
        
        context_data['bottom_detection'] = (np.where(context_data['mvrv_z_score'] <= 0, 1, 0) * 
                                            np.where(context_data['mayer_multiple'] <= 0.8, 1, 0) *
                                            np.where(context_data['price_usd_close'] <= context_data['price_realized_usd'], 1, 0) *
                                            np.where(context_data['net_unrealized_profit_loss_account_based'] <= 0, 1, 0) *
                                            np.where(context_data['puell_multiple'] <= 0.5, 1, 0) *
                                            np.where(context_data['dormancy_flow'] <= 200000, 1, 0)) * context_data['price_usd_close']

        # Apply context determination
        context_data['context'] = context_data.apply(self.determine_context, axis=1)
        context_data['context'].fillna(method='ffill', inplace=True)
        context_data.drop(columns=['top_detection', 'bottom_detection'], inplace=True)
        context_data = context_data[context_data.index >= start]
        
        return context_data

    @staticmethod
    def determine_context(row):
        if row['bottom_detection'] != 0 and (row['top_detection'] == 0 or row.name < row.index[row['top_detection'] != 0].max()):
            return int(1)
        elif row['top_detection'] != 0 and (row['bottom_detection'] == 0 or row.name < row.index[row['bottom_detection'] != 0].max()):
            return int(0)
        else:
            return np.nan

    def merge_data(self, trigger_data, context_data):
        # Ensure proper alignment of the data indices before merging
        trigger_data.reset_index(inplace=True)
        context_data.reset_index(inplace=True)
        context_data['t'] = context_data['t'].dt.tz_localize(None)  # Ensure no timezone differences
        
        # Merge the datasets
        full_data = pd.merge_asof(trigger_data, context_data[['t', 'context']], on='t', direction='forward')
        full_data['context'] = full_data['context'].ffill()
        full_data.set_index('t', inplace=True)

        return full_data


In [5]:
class TradingEnvironment:
    def __init__(self, data, window_size, scaler=StandardScaler()):
        self.data = data
        self.window_size = window_size
        self.cash = 10000
        self.initial_cash = 10000  # Starting cash
        self.bitcoin = 0  # Bitcoin holding, positive for long, negative for short
        self.entry_price = 0  # Price at which the last trade was executed
        self.net_worth = self.cash  # Net worth initialized to starting cash
        self.scaler = scaler  # Optional scaler
        self.scaled_data = pd.DataFrame(scaler.fit_transform(self.data), columns=self.data.columns, index=self.data.index)
        self.current_step = 0
        self.position_state = 0  # 0: long, 1: cash, 2: short
        self.reset()

    def reset(self):
        self.current_step = self.window_size
        self.cash = 10000
        self.bitcoin = 0
        self.entry_price = 0
        self.net_worth = self.cash
        self.position_state = 0
        return self._get_state()

    def _get_state(self):
        state_columns = [col for col in self.scaled_data.columns if col != 'context']
        state = self.scaled_data[state_columns].iloc[self.current_step - self.window_size:self.current_step].values
        return state.astype(float)

    def step(self, action):
        if self.current_step + 1 >= len(self.data):
            raise IndexError("Attempted to step beyond the data boundary.")

        current_price = self.data.iloc[self.current_step]['price_usd_close']
        
        if action == 1:  # Long
            if self.position_state != 1 and self.cash > 0:
                self.bitcoin = self.cash / current_price
                self.entry_price = current_price
                self.cash = 0
                self.position_state = 1

        elif action == 0:  # Cash
            if self.position_state == 1:  # Closing long
                self.cash = self.bitcoin * current_price
                self.bitcoin = 0
            elif self.position_state == -1:  # Closing short
                profit = -self.bitcoin * (self.entry_price - current_price)
                self.cash += profit
                self.bitcoin = 0
            self.position_state = 0

        elif action == -1:  # Short
            if self.position_state != -1 and self.cash > 0:
                self.bitcoin = -self.cash / current_price  # Short position
                self.entry_price = current_price
                self.cash = 0
                self.position_state = -1

        
        self.net_worth = self.cash + (self.bitcoin * current_price if self.bitcoin > 0 else 0)
        
        # Calculate reward based on net worth change, not just final amount
        reward = self.net_worth - self.initial_cash
        
        if reward > 0:
            reward = 1
        elif reward < 0:
            reward = -1
        else:
            reward = 0
        self.current_step += 1
        done = self.net_worth <= 0 or self.current_step >= len(self.data) - 1

        return self._get_state(), reward, done


In [21]:
class DuelingQNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(DuelingQNetwork, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.fc1 = nn.Linear(state_size, 256)
        self.fc2 = nn.Linear(256, 256)
        
        # Value stream
        self.value_fc = nn.Linear(256, 1)
        
        # Advantage stream
        self.advantage_fc = nn.Linear(256, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        
        value = self.value_fc(x)
        advantage = self.advantage_fc(x)
        
        q_value = value + (advantage - advantage.mean(dim=1, keepdim=True))
        return q_value.squeeze(1)

class DuelingDQNAgent:
    def __init__(self, state_size, action_size):    
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

    def _build_model(self):
        return DuelingQNetwork(self.state_size, self.action_size)

    def update_target_model(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state, context):
        # Exploration: Random actions based on context
        if random.random() <= self.epsilon:
            if context == 1:
                return random.choice([0, 1])  # Long or Cash
            elif context == 0:
                return random.choice([-1, 0])  # Cash or Short

        # Prepare the state for the model
        state = torch.FloatTensor(state).reshape(-1, self.state_size).unsqueeze(0)

        # Exploitation: Choose the best action based on the Q-values from the model
        act_values = self.model(state)  # Retrieve Q-values from the model
        print(act_values)
        if context == 1:
            # Only consider Long or Cash
            return torch.argmax(act_values[:, :2], 1).item()  # Index 0 for Cash, 1 for Long
        elif context == 0:
            # Only consider Cash or Short
            action = torch.argmax(act_values[:, 1:], 1).item()  # This will be 0 for Long, 1 for Short
            
            if action == 0:
                return 0  # Cash
            else:
                return -1  # Short

        # If no context restriction, choose the best overall action
        return torch.argmax(act_values, 1).item()  # Choose the action with the highest Q-value


    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return

        minibatch = random.sample(self.memory, batch_size)

        for state, action, reward, next_state, done in minibatch:
            
            state = torch.FloatTensor(state).reshape(-1, self.state_size)
            next_state = torch.FloatTensor(next_state).reshape(-1, self.state_size)
            
            current_qs = self.model(state)
            next_qs_target = self.target_model(next_state)
            
            max_next_qs = torch.max(next_qs_target, dim=1)[0]
            target_qs = current_qs.clone().detach()
            targets = reward + self.gamma * max_next_qs * (1 - done)  # Done flag to zero out terminal states
            
            target_qs[0, action] = targets
            
            loss = nn.MSELoss()(current_qs, target_qs)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        # Epsilon decay
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


    def load(self, name):
        self.model.load_state_dict(torch.load(name))

    def save(self, name):
        torch.save(self.model.state_dict(), name)





In [7]:
def train_dqn(agent, env, repetitions, batch_size):
    for rep in range(repetitions):
        state = env.reset()
        total_reward = 0
        steps = 0
        while True:
            context = env.data.iloc[env.current_step]['context']
            action = agent.act(state, context)
            next_state, reward, done = env.step(action)
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward
            steps += 1

            print(f"Rep: {rep}, Step: {steps}, Action: {action}, Reward: {reward}, Done: {done}")

            if done:
                print(f"Training completed for Repetition {rep+1}. Total Steps: {steps}, Total Reward: {total_reward}")
                break

            if len(agent.memory) > batch_size:
                agent.replay(batch_size)


In [344]:
full_data = pd.read_csv('/Users/valter.rebelo/alfa_trader/ALFA_TRADER/data/full_data_binance.csv')
full_data

,t,open,high,low,close,rsi_ssr_smoothed,rsi_ssr_smoothed_median,context
0,2018-09-30 00:00:00,6597.66,6597.66,6557.70,6577.30,47.884933,47.435209,0.0
1,2018-09-30 01:00:00,6577.31,6607.43,6570.20,6595.48,47.890065,47.435631,0.0
2,2018-09-30 02:00:00,6595.47,6602.00,6570.00,6575.16,47.894873,47.436323,0.0
3,2018-09-30 03:00:00,6574.54,6584.24,6570.00,6580.00,47.899770,47.437415,0.0
4,2018-09-30 04:00:00,6576.34,6581.99,6552.44,6552.49,47.904119,47.438184,0.0
...,...,...,...,...,...,...,...,...
50399,2024-06-29 23:00:00,60970.00,61059.37,60940.00,60986.68,47.508964,48.264516,1.0
50400,2024-06-30 00:00:00,60986.68,61078.01,60922.00,61024.55,47.504625,48.253343,1.0
50401,2024-06-30 01:00:00,61024.55,61043.44,60930.03,60961.99,47.500037,48.242261,1.0
50402,2024-06-30 02:00:00,60961.99,60961.99,60740.94,60834.27,47.495165,48.232336,1.0


In [ ]:
data_manager = DataManager()
strategy = TradingStrategy(data_manager)

#start = '2018-09-30' # data onde temos o primeiro indicador point do SRS 
#end = '2024-06-30'

#context_data = strategy.compute_context('2011-08-10', end) # é preciso de mais data points para calcular o contexto de mercado
#trigger_data = strategy.compute_triggers(start, end)
#full_data = strategy.merge_data(trigger_data, context_data)

full_data = pd.read_csv('/Users/valter.rebelo/alfa_trader/ALFA_TRADER/data/full_data_binance.csv')
env = TradingEnvironment(full_data, window_size=10) # must be mindful of window size (days)


In [22]:
env.reset()
state_size = env._get_state().shape[0] * env._get_state().shape[1]
print(state_size)
action_size = 3  # Long, Cash, Short
agent = DuelingDQNAgent(state_size, action_size)

30


In [23]:
train_dqn(agent=agent, env=env, repetitions=3, batch_size=32)


Rep: 0, Step: 1, Action: -1, Reward: -1, Done: True
Training completed for Repetition 1. Total Steps: 1, Total Reward: -1
Rep: 1, Step: 1, Action: -1, Reward: -1, Done: True
Training completed for Repetition 2. Total Steps: 1, Total Reward: -1
Rep: 2, Step: 1, Action: 0, Reward: 0, Done: False
Rep: 2, Step: 2, Action: -1, Reward: -1, Done: True
Training completed for Repetition 3. Total Steps: 2, Total Reward: -1


In [533]:
class Backtester:
    
    def __init__(self, environment, agent, initial_capital=10000):
        self.data = environment
        self.agent = agent
        self.window_size = environment.window_size
        self.initial_capital = initial_capital
        self.results = {}
        self.run_backtest()

    def run_backtest(self):
        self.data.data['position'] = 0
        self.cash = self.initial_capital
        self.bitcoin = 0
        self.net_worth = self.cash
        self.positions = []

        for i in range(self.window_size, len(self.data)):
            state = self.data.iloc[i-self.window_size:i].values
            context = self.data.iloc[i]['context']
            action = self.agent.act(state, context)

            current_price = self.data.iloc[i]['price_usd_close']

            if context == 1:
                if action == 0 and self.cash > 0:  # Long
                    self.bitcoin += self.cash / current_price
                    self.cash = 0
                    self.positions.append((self.data.index[i], 'long'))
                elif action == 1:  # Cash
                    pass
            elif context == 0:
                if action == 1 and self.cash > 0:  # Cash
                    pass
                elif action == 2 and self.bitcoin > 0:  # Short
                    self.cash += self.bitcoin * current_price
                    self.bitcoin = 0
                    self.positions.append((self.data.index[i], 'short'))

            self.net_worth = self.cash + self.bitcoin * current_price
            self.data.loc[self.data.index[i], 'net_worth'] = self.net_worth

        self.calculate_statistics()

    def calculate_statistics(self):
        self.results['Start'] = self.data.index.min()
        self.results['End'] = self.data.index.max()
        self.results['Duração'] = self.results['End'] - self.results['Start']
        self.results['Tempo de Exposição [%]'] = np.mean(self.data['position'] != 0) * 100
        self.results['Saldo Final [$]'] = self.data['net_worth'].iloc[-1]
        self.results['Pico de Valor [$]'] = self.data['net_worth'].max()
        self.results['Retorno AlfaTrader [%]'] = ((self.results['Saldo Final [$]'] / self.initial_capital - 1) * 100)
        self.results['Retorno Buy & Hold [%]'] = ((self.data['price_usd_close'].iloc[-1] / self.data['price_usd_close'].iloc[0] - 1) * 100)
        self.results['Retorno (Ann.) [%]'] = self.results['Retorno AlfaTrader [%]'] / (self.results['Duração'].days / 365.25)
        self.results['Volatilidade (Ann.) [%]'] = self.data['strategy_return'].std() * np.sqrt(365*24) * 100

        # Filter to include only non-zero returns or when the strategy is active
        active_returns = self.data['strategy_return'][self.data['strategy_return'] != 0]

        # Calculate Sharpe Ratio using non-zero returns
        if active_returns.empty:
            self.results['Sharpe Ratio'] = 0  # Handle case where there are no active returns
        else:
            self.results['Sharpe Ratio'] = (active_returns.mean() / self.data['strategy_return'].std()) * np.sqrt(8760)

        # Calculate negative returns for the downside deviation
        negative_returns = self.data['strategy_return'][self.data['strategy_return'] < 0]

        # Calculate Sortino Ratio using non-zero returns and downside deviation
        if not negative_returns.empty and not active_returns.empty:
            mean_active_returns = active_returns.mean()
            downside_deviation = negative_returns.std()
            annual_factor = np.sqrt(8760)  # Assuming 24/7 trading for crypto markets
            self.results['Sortino Ratio'] = (mean_active_returns / downside_deviation) * annual_factor
        else:
            self.results['Sortino Ratio'] = 0  # Handle case where there are no negative returns or active returns

        # Trading statistics
        trades = self.data['position'].diff().fillna(0) != 0
        self.results['# Trades'] = trades.sum()

    def get_results(self):
        return pd.DataFrame([self.results])




In [250]:
class StrategyVisualizer:
    def __init__(self, data):
        self.data = data

    def plot_results(self):
        # Create subplots: one for portfolio values and one for BTC price with signals
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                            vertical_spacing=0.1, subplot_titles=('Valor do Portfólio', 'Preço BTC com Sinais'))

        # Plotting portfolio values: Strategy vs Buy & Hold
        fig.add_trace(go.Scatter(x=self.data.index, y=self.data['strategy_cumulative_return'], name='AlfaTrader ®', line=dict(color='green')), row=1, col=1)
        #fig.add_trace(go.Scatter(x=self.data.index, y=self.data['price_usd_close'], name='BTC', line=dict(color='blue')), row=1, col=1)
        fig.add_trace(go.Scatter(x=self.data.index, y=self.data['market_portfolio_cumulative_return'], name='Comprar e Segurar', line=dict(color='blue')), row=1, col=1)
        # BTC Price and action markers
        fig.add_trace(go.Scatter(x=self.data.index, y=self.data['price_usd_close'], name='Preço BTC', line=dict(color='black')), row=2, col=1)

        # Adding markers only at points of action change
        action_changes = self.data[self.data['action'] != self.data['action'].shift(1)]
        buys = action_changes[action_changes['action'] == 'long']
        cash = action_changes[action_changes['action'] == 'cash']
        shorts = action_changes[action_changes['action'] == 'short']

        fig.add_trace(go.Scatter(x=buys.index, y=buys['price_usd_close'], mode='markers', marker=dict(color='#4AD811', size=8, symbol='triangle-up'), name='Compra'), row=2, col=1)
        fig.add_trace(go.Scatter(x=cash.index, y=cash['price_usd_close'], mode='markers', marker=dict(color='#FFCC2D', size=8, symbol='square'), name='Caixa'), row=2, col=1)
        fig.add_trace(go.Scatter(x=shorts.index, y=shorts['price_usd_close'], mode='markers', marker=dict(color='#C70039', size=8, symbol='triangle-down'), name='Venda'), row=2, col=1)

        # Update layout
        fig.update_layout(height=800, width=1000, title_text="", hovermode="x unified", plot_bgcolor='white', paper_bgcolor='white', font=dict(color='black'))
        fig.show()



    def plot_btc_with_signals(self):
        # Create a figure with custom subplots
        fig = make_subplots(specs=[[{"secondary_y": True}]])

        # Plotting BTC price as a line graph
        fig.add_trace(go.Scatter(x=self.data.index, y=self.data['price_usd_close'], name='Preço BTC', line=dict(color='blue')), secondary_y=False)

        # Ensure that only points where detection is marked as '1' have non-zero heights
        # Top detections as green bars
        top_heights = self.data['price_usd_close'].where(self.data['top_detection'] != 0, 0)
        fig.add_trace(go.Bar(x=self.data.index, y=top_heights, name='Detecção de Topo', marker_color='green'), secondary_y=False)

        # Bottom detections as red bars
        bottom_heights = self.data['price_usd_close'].where(self.data['bottom_detection'] != 0, 0)
        fig.add_trace(go.Bar(x=self.data.index, y=bottom_heights, name='Detecção de Fundo', marker_color='red'), secondary_y=False)

        # Update layout for aesthetics and readability
        fig.update_layout(
            title='Preço BTC com Indicadores de Topo e Fundo',
            xaxis_title='',
            yaxis_title='Preço BTC',
            legend_title='Legenda',
            hovermode='x unified',
            plot_bgcolor='white',
            paper_bgcolor='white',
            font=dict(color='black')
        )

        # Adjust the y-axes visual appearance
        fig.update_yaxes(title_text="Preço BTC", secondary_y=False)

        fig.show()


In [51]:
visualizer = StrategyVisualizer(final_data)
#visualizer.plot_btc_with_signals()
visualizer.plot_results()
